In [1]:
const async = require('async');
const fs = require('fs');

# Each
coll 리스트에 반복자 함수를 모두 병렬로 적용

for in 반복문과 같이 매개변수로 받은 것을 이용해 배열 or 객체 요소를 검사

반복자 함수는 리스트가 완료되면 콜백하지만, error 발생하면 메인 콜백 즉시 호출

병렬이므로 순서 보장X


In [ ]:
// dir1은 file1.txt, file2.txt가 포함된 디렉토리입니다.
// dir2는 file3.txt, file4.txt가 포함된 디렉토리입니다.
// dir3은 file5.txt가 포함된 디렉토리입니다.
// dir4는 존재하지 않습니다.

const fileList = [ 'dir1/file2.txt', 'dir2/file3.txt', 'dir3/file5.txt'];
const withMissingFileList = ['dir1/file1.txt', 'dir4/file2.txt'];

// 파일을 삭제하는 비동기 함수
const deleteFile = function(file, callback) {
    fs.unlink(file, callback);
};

// 콜백 사용
async.each(fileList, deleteFile, function(err) {
    if( err ) {
        console.log(err);
    } else {
        console.log('모든 파일이 성공적으로 삭제되었습니다');
    }
});

// 에러 처리
async.each(withMissingFileList, deleteFile, function(err){
    console.log(err);
    // [ Error: ENOENT: no such file or directory ]
    // dir4/file2.txt가 존재하지 않기 때문입니다.
    // dir1/file1.txt는 삭제되었을 수 있습니다.
});

// 프로미스 사용
async.each(fileList, deleteFile)
.then( () => {
    console.log('모든 파일이 성공적으로 삭제되었습니다');
}).catch( err => {
    console.log(err);
});

// 에러 처리
async.each(fileList, deleteFile)
.then( () => {
    console.log('모든 파일이 성공적으로 삭제되었습니다');
}).catch( err => {
    console.log(err);
    // [ Error: ENOENT: no such file or directory ]
    // dir4/file2.txt가 존재하지 않기 때문입니다.
    // dir1/file1.txt는 삭제되었을 수 있습니다.
});

// async/await 사용
async () => {
    try {
        await async.each(files, deleteFile);
    }
    catch (err) {
        console.log(err);
    }
}

// 에러 처리
async () => {
    try {
        await async.each(withMissingFileList, deleteFile);
    }
    catch (err) {
        console.log(err);
        // [ Error: ENOENT: no such file or directory ]
        // dir4/file2.txt가 존재하지 않기 때문입니다.
        // dir1/file1.txt는 삭제되었을 수 있습니다.
    }
}

[AsyncFunction (anonymous)]

모든 파일이 성공적으로 삭제되었습니다


# Every
coll의 모든 요소가 비동기 테스트를 만족하면 true 반환
하나라도 만족하지 못하면 false이므로 반복 호출 중 false 반환하면 메인 콜백 즉시 호출

In [1]:
// dir1은 file1.txt, file2.txt가 포함된 디렉토리입니다.
// dir2는 file3.txt, file4.txt가 포함된 디렉토리입니다.
// dir3은 file5.txt가 포함된 디렉토리입니다.
// dir4는 존재하지 않습니다.

const fileList = ['dir1/file1.txt','dir2/file3.txt','dir3/file5.txt'];
const withMissingFileList = ['file1.txt','file2.txt','file4.txt'];

// 파일 존재 여부를 확인하는 비동기 함수
function fileExists(file, callback) {
   fs.access(file, fs.constants.F_OK, (err) => {
       callback(null, !err);
   });
}

// 콜백 사용
async.every(fileList, fileExists, function(err, result) {
    console.log(result);
    // true
    // 모든 파일이 존재하므로 result는 true입니다.
});

async.every(withMissingFileList, fileExists, function(err, result) {
    console.log(result);
    // false
    // 모든 파일이 존재하지 않으므로 result는 false입니다.
});

// 프로미스 사용
async.every(fileList, fileExists)
.then( result => {
    console.log(result);
    // true
    // 모든 파일이 존재하므로 result는 true입니다.
}).catch( err => {
    console.log(err);
});

async.every(withMissingFileList, fileExists)
.then( result => {
    console.log(result);
    // false
    // 모든 파일이 존재하지 않으므로 result는 false입니다.
}).catch( err => {
    console.log(err);
});

// async/await 사용
async () => {
    try {
        let result = await async.every(fileList, fileExists);
        console.log(result);
        // true
        // 모든 파일이 존재하므로 result는 true입니다.
    }
    catch (err) {
        console.log(err);
    }
}

async () => {
    try {
        let result = await async.every(withMissingFileList, fileExists);
        console.log(result);
        // false
        // 모든 파일이 존재하지 않으므로 result는 false입니다.
    }
    catch (err) {
        console.log(err);
    }
}


false
true


# Filter

배열의 요소를 순회하면서 콜백함수 사용해 원하는 조건에 따라 필터링

병렬로 수행되지만 결과 배열은 오리지널과 같은 순서로 반환됨

In [2]:
// dir1은 file1.txt, file2.txt가 포함된 디렉토리입니다.
// dir2는 file3.txt, file4.txt가 포함된 디렉토리입니다.
// dir3은 file5.txt가 포함된 디렉토리입니다.

const files = ['dir1/file1.txt','dir2/file3.txt','dir3/file6.txt'];

// 파일 존재 여부를 확인하는 비동기 함수
function fileExists(file, callback) {
   fs.access(file, fs.constants.F_OK, (err) => {
       callback(null, !err);
   });
}

// 콜백 사용
async.filter(files, fileExists, function(err, results) {
   if(err) {
       console.log(err);
   } else {
       console.log(results);
       // [ 'dir1/file1.txt', 'dir2/file3.txt' ]
       // results는 이제 존재하는 파일의 배열입니다.
   }
});

// 프로미스 사용
async.filter(files, fileExists)
.then(results => {
    console.log(results);
    // [ 'dir1/file1.txt', 'dir2/file3.txt' ]
    // results는 이제 존재하는 파일의 배열입니다.
}).catch(err => {
    console.log(err);
});

// async/await 사용
async () => {
    try {
        let results = await async.filter(files, fileExists);
        console.log(results);
        // [ 'dir1/file1.txt', 'dir2/file3.txt' ]
        // results는 이제 존재하는 파일의 배열입니다.
    }
    catch (err) {
        console.log(err);
    }
}

[AsyncFunction (anonymous)]

[ 'dir1/file1.txt', 'dir2/file3.txt' ]
[ 'dir1/file1.txt', 'dir2/file3.txt' ]


# groupBy

배열의 값을 묶거나 분류할 때 활용할 수 있는 함수

배열 내의 모든 요소 각각에 대해 주어진 함수를 호출한 결과를 모아 새로운 배열로 반환

병렬로 적용되어 순서보장X

하지만 결과의 각 키에 대한 값은 원래와 같은 순서

In [1]:
// dir1은 file1.txt, file2.txt가 포함된 디렉토리입니다.
// dir2는 file3.txt, file4.txt가 포함된 디렉토리입니다.
// dir3은 file5.txt가 포함된 디렉토리입니다.
// dir4는 존재하지 않습니다.

const files = ['dir1/file1.txt','dir2','dir4']

// 파일 타입을 none, file, directory로 감지하는 비동기 함수
function detectFile(file, callback) {
    fs.stat(file, function(err, stat) {
        if (err) {
            return callback(null, 'none');
        }
        callback(null, stat.isDirectory() ? 'directory' : 'file');
    });
}

// 콜백 사용
async.groupBy(files, detectFile, function(err, result) {
    if(err) {
        console.log(err);
    } else {
        console.log(result);
        // {
        //     file: [ 'dir1/file1.txt' ],
        //     none: [ 'dir4' ],
        //     directory: [ 'dir2']
        // }
        // result는 파일을 타입별로 그룹화한 객체입니다.
    }
});

// 프로미스 사용
async.groupBy(files, detectFile)
.then(result => {
    console.log(result);
    // {
    //     file: [ 'dir1/file1.txt' ],
    //     none: [ 'dir4' ],
    //     directory: [ 'dir2']
    // }
    // result는 파일을 타입별로 그룹화한 객체입니다.
}).catch(err => {
    console.log(err);
});

// async/await 사용
async () => {
    try {
        let result = await async.groupBy(files, detectFile);
        console.log(result);
        // {
        //     file: [ 'dir1/file1.txt' ],
        //     none: [ 'dir4' ],
        //     directory: [ 'dir2']
        // }
        // result는 파일을 타입별로 그룹화한 객체입니다.
    }
    catch (err) {
        console.log(err);
    }
}


[AsyncFunction (anonymous)]

{ file: [ 'dir1/file1.txt' ], directory: [ 'dir2' ], none: [ 'dir4' ] }
{ file: [ 'dir1/file1.txt' ], directory: [ 'dir2' ], none: [ 'dir4' ] }


# map

배열을 순회해서 각 요소를 콜백 함수로 적용해서 처리해 모은 새로운 배열을 반환하기 위한 함수

반복자가 Error를 콜백으로 전달하면 지도함수의 메인 콜백 즉시 오류와 함께 호출

원본 배열은 변경하지 않으면서 해당 배열 요소에 대한 규칙적인 새로운 배열 요소를 생성할 때 사용

순서대로 완료 보장X, 결과 배열은 원래 배열과 같은 순서 유지

In [2]:
// file1.txt는 크기가 1000바이트인 파일입니다.
// file2.txt는 크기가 2000바이트인 파일입니다.
// file3.txt는 크기가 3000바이트인 파일입니다.
// file4.txt는 존재하지 않습니다.

const fileList = ['file1.txt','file2.txt','file3.txt'];
const withMissingFileList = ['file1.txt','file2.txt','file4.txt'];

// 파일 크기를 바이트 단위로 반환하는 비동기 함수
function getFileSizeInBytes(file, callback) {
    fs.stat(file, function(err, stat) {
        if (err) {
            return callback(err);
        }
        callback(null, stat.size);
    });
}

// 콜백 사용
async.map(fileList, getFileSizeInBytes, function(err, results) {
    if (err) {
        console.log(err);
    } else {
        console.log(results);
        // results는 이제 각 파일의 크기를 바이트 단위로 나타내는 배열입니다.
        // [ 1000, 2000, 3000]
    }
});

// 에러 처리
async.map(withMissingFileList, getFileSizeInBytes, function(err, results) {
    if (err) {
        console.log(err);
        // [ Error: ENOENT: no such file or directory ]
    } else {
        console.log(results);
    }
});

// 프로미스 사용
async.map(fileList, getFileSizeInBytes)
.then(results => {
    console.log(results);
    // results는 이제 각 파일의 크기를 바이트 단위로 나타내는 배열입니다.
    // [ 1000, 2000, 3000]
}).catch(err => {
    console.log(err);
});

// 에러 처리
async.map(withMissingFileList, getFileSizeInBytes)
.then(results => {
    console.log(results);
}).catch(err => {
    console.log(err);
    // [ Error: ENOENT: no such file or directory ]
});

// async/await 사용
async () => {
    try {
        let results = await async.map(fileList, getFileSizeInBytes);
        console.log(results);
        // results는 이제 각 파일의 크기를 바이트 단위로 나타내는 배열입니다.
        // [ 1000, 2000, 3000]
    }
    catch (err) {
        console.log(err);
    }
}

// 에러 처리
async () => {
    try {
        let results = await async.map(withMissingFileList, getFileSizeInBytes);
        console.log(results);
    }
    catch (err) {
        console.log(err);
        // [ Error: ENOENT: no such file or directory ]
    }
}


[AsyncFunction (anonymous)]

[Error: ENOENT: no such file or directory, stat 'c:\Users\신초은\Desktop\Nodejs\server_basic\Jupyter\file1.txt'] {
  errno: -4058,
  code: 'ENOENT',
  syscall: 'stat',
  path: 'c:\\Users\\신초은\\Desktop\\Nodejs\\server_basic\\Jupyter\\file1.txt'
}
[Error: ENOENT: no such file or directory, stat 'c:\Users\신초은\Desktop\Nodejs\server_basic\Jupyter\file1.txt'] {
  errno: -4058,
  code: 'ENOENT',
  syscall: 'stat',
  path: 'c:\\Users\\신초은\\Desktop\\Nodejs\\server_basic\\Jupyter\\file1.txt'
}
[Error: ENOENT: no such file or directory, stat 'c:\Users\신초은\Desktop\Nodejs\server_basic\Jupyter\file1.txt'] {
  errno: -4058,
  code: 'ENOENT',
  syscall: 'stat',
  path: 'c:\\Users\\신초은\\Desktop\\Nodejs\\server_basic\\Jupyter\\file1.txt'
}
[Error: ENOENT: no such file or directory, stat 'c:\Users\신초은\Desktop\Nodejs\server_basic\Jupyter\file1.txt'] {
  errno: -4058,
  code: 'ENOENT',
  syscall: 'stat',
  path: 'c:\\Users\\신초은\\Desktop\\Nodejs\\server_basic\\Jupyter\\file1.txt'
}


# some

배열에서 적어도 어떤 하나의 요소가 비동기 테스트를 통과하면 true 반환

만약 어떤 반복자 호출이 true 반환 -> 즉시 메인 콜백 호출

In [ ]:
// dir1은 file1.txt, file2.txt가 포함된 디렉토리입니다.
// dir2는 file3.txt, file4.txt가 포함된 디렉토리입니다.
// dir3은 file5.txt가 포함된 디렉토리입니다.
// dir4는 존재하지 않습니다.

// 파일 존재 여부를 확인하는 비동기 함수
function fileExists(file, callback) {
    fs.access(file, fs.constants.F_OK, (err) => {
        callback(null, !err);
    });
 }
 
 // 콜백 사용
 async.some(['dir1/missing.txt','dir2/missing.txt','dir3/file5.txt'], fileExists,
    function(err, result) {
        console.log(result);
        // true
        // 목록에 있는 일부 파일이 존재하므로 result는 true입니다.
    }
 );
 
 async.some(['dir1/missing.txt','dir2/missing.txt','dir4/missing.txt'], fileExists,
    function(err, result) {
        console.log(result);
        // false
        // 목록에 있는 파일이 하나도 존재하지 않으므로 result는 false입니다.
    }
 );
 
 // 프로미스 사용
 async.some(['dir1/missing.txt','dir2/missing.txt','dir3/file5.txt'], fileExists)
 .then(result => {
     console.log(result);
     // true
     // 목록에 있는 일부 파일이 존재하므로 result는 true입니다.
 }).catch(err => {
     console.log(err);
 });
 
 async.some(['dir1/missing.txt','dir2/missing.txt','dir4/missing.txt'], fileExists)
 .then(result => {
     console.log(result);
     // false
     // 목록에 있는 파일이 하나도 존재하지 않으므로 result는 false입니다.
 }).catch(err => {
     console.log(err);
 });
 
 // async/await 사용
 async () => {
     try {
         let result = await async.some(['dir1/missing.txt','dir2/missing.txt','dir3/file5.txt'], fileExists);
         console.log(result);
         // true
         // 목록에 있는 일부 파일이 존재하므로 result는 true입니다.
     }
     catch (err) {
         console.log(err);
     }
 }
 
 async () => {
     try {
         let result = await async.some(['dir1/missing.txt','dir2/missing.txt','dir4/missing.txt'], fileExists);
         console.log(result);
         // false
         // 목록에 있는 파일이 하나도 존재하지 않으므로 result는 false입니다.
     }
     catch (err) {
         console.log(err);
     }
 }
 

[AsyncFunction (anonymous)]

false
true
true
false
